In [1]:
#Importing libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.weightstats import DescrStatsW
from statsmodels.formula.api import ols

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from scipy.spatial import distance


## Data Import

#### Importing Demography DataSets Year 2011-2018 

In [3]:
file_paths1 = [
    '/Users/alexandragort/Documents/BA/Data/DEMOGRAPHY_2011-2012.XPT',
    '/Users/alexandragort/Documents/BA/Data/DEMOGRAPHY_2013-2014.XPT',
    '/Users/alexandragort/Documents/BA/Data/DEMOGRAPHY_2015-2016.XPT',
    '/Users/alexandragort/Documents/BA/Data/DEMOGRAPHY_2017-2018.XPT'
]

dem_data = {}

for path1 in file_paths1:
    # Extract year range from the file path
    year_range1 = path1.split('_')[-1].split('.')[0]  # Extracts '2011-2012', '2013-2014', etc.
    
    df = pd.read_sas(path1, format='xport')
  
    columns_to_keep = ['SEQN', 'SDDSRVYR', 'RIAGENDR', 'RIDAGEYR','INDHHIN2', 'RIDRETH3', 'DMDCITZN', 'SDMVPSU', 'SDMVSTRA', 'WTINT2YR', 'WTMEC2YR']
    # Dropping irrelevant columns
    df_filtered = df[columns_to_keep]
    
    dem_data[year_range1] = df_filtered

# Example to access data for a specific year range
dem_data['2011-2012'].head()


,SEQN,SDDSRVYR,RIAGENDR,RIDAGEYR,INDHHIN2,RIDRETH3,DMDCITZN,SDMVPSU,SDMVSTRA,WTINT2YR,WTMEC2YR
0,62161.0,7.0,1.0,22.0,14.0,3.0,1.0,1.0,91.0,102641.406474,104236.582554
1,62162.0,7.0,2.0,3.0,4.0,1.0,1.0,3.0,92.0,15457.736897,16116.354010
2,62163.0,7.0,1.0,14.0,15.0,6.0,1.0,3.0,90.0,7397.684828,7869.485117
3,62164.0,7.0,2.0,44.0,8.0,3.0,1.0,1.0,94.0,127351.373299,127965.226204
4,62165.0,7.0,2.0,14.0,4.0,4.0,1.0,2.0,90.0,12209.744980,13384.042162


In [4]:
# Print column names to check for weight variables
print(dem_data['2017-2018'].columns)

Index(['SEQN', 'SDDSRVYR', 'RIAGENDR', 'RIDAGEYR', 'INDHHIN2', 'RIDRETH3',
       'DMDCITZN', 'SDMVPSU', 'SDMVSTRA', 'WTINT2YR', 'WTMEC2YR'],
      dtype='object')


As can be seen the  weights and design variables (Strata and Sampling design units) 'WTINT2YR', 'WTMEC2YR', 'SDMVPSU', 'SDMVSTRA' are displayed in the demography dataset. They can be found across all the four cycles (consitency ensured). So before adjusting the weights I will merge the three datasets (oral health, health insurance coverage and demography) per cycle together after importing all of them.

#### Combine all four cylces and adjust the weights

In [5]:
combined_dem_data = pd.concat([dem_data['2011-2012'], dem_data['2013-2014'], dem_data['2015-2016'], dem_data['2017-2018']])

# Calculating the new weights
# The new weight are calculated by dividing the original weight by 4 (like advised in NHANES analytic guidelines)
combined_dem_data['WTINT8YR'] = combined_dem_data['WTINT2YR'] / 4
combined_dem_data['WTMEC8YR'] = combined_dem_data['WTMEC2YR'] / 4

# Dropping the original weight columns 
combined_dem_data.drop(columns=['WTINT2YR', 'WTMEC2YR'], inplace=True)


combined_dem_data.to_csv('nhanes_2011_2018_combined.csv', index=False)

In [6]:
combined_dem_data = pd.read_csv('/Users/alexandragort/Documents/BA/nhanes_2011_2018_combined.csv')

combined_dem_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39156 entries, 0 to 39155
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      39156 non-null  float64
 1   SDDSRVYR  39156 non-null  float64
 2   RIAGENDR  39156 non-null  float64
 3   RIDAGEYR  39156 non-null  float64
 4   INDHHIN2  38106 non-null  float64
 5   RIDRETH3  39156 non-null  float64
 6   DMDCITZN  39142 non-null  float64
 7   SDMVPSU   39156 non-null  float64
 8   SDMVSTRA  39156 non-null  float64
 9   WTINT8YR  39156 non-null  float64
 10  WTMEC8YR  39156 non-null  float64
dtypes: float64(11)
memory usage: 3.3 MB


#### Importing Healthinsurance Coverage DataSets Year 2011-2018 and dropping irrelevant variables

In [7]:
file_paths2 = [
    '/Users/alexandragort/Documents/BA/Data/HICOVERAGE_2011-2012.XPT',
    '/Users/alexandragort/Documents/BA/Data/HICOVERAGE_2013-2014.XPT',
    '/Users/alexandragort/Documents/BA/Data/HICOVERAGE_2015-2016.XPT',
    '/Users/alexandragort/Documents/BA/Data/HICOVERAGE_2017-2018.XPT'
]

hic_data = {}

for path2 in file_paths2:

    year_range2 = path2.split('_')[-1].split('.')[0]  # Extracts '2003-2004', '2005-2006', etc.
    
    hic_data[year_range2] = pd.read_sas(path2, format='xport')

# Example to access data for a specific year range
hic_data['2011-2012'].head()  

,SEQN,HIQ011,HIQ031A,HIQ031B,HIQ031C,HIQ031D,HIQ031E,HIQ031F,HIQ031G,HIQ031H,HIQ031I,HIQ031J,HIQ031AA,HIQ260,HIQ105,HIQ270,HIQ210
0,62161.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
1,62162.0,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
2,62163.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
3,62164.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
4,62165.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop cols that I dont need for the analysis
columns_to_drop2 = ['HIQ260', 'HIQ105', 'HIQ270', 'HIQ210', 'HIQ031AA']

for year_range2 in hic_data:
    hic_data[year_range2] = hic_data[year_range2].drop(columns_to_drop2, axis=1)



In [9]:
# Cheking for nan values over all the rows this will be relevant for later where missing values are handled
columns_to_check2 = ['HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E', 
                     'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'HIQ031J']


nan_rows_count = {}

# Iterate over each DataFrame and compute the number of rows with all NaN values
for year_range, df in hic_data.items():
    
    valid_columns = [col for col in columns_to_check2 if col in df.columns]
    
    if valid_columns:  
        nan_rows_count[year_range] = df[valid_columns].isna().all(axis=1).sum()

nan_rows_count

{'2011-2012': 1727, '2013-2014': 1592, '2015-2016': 1335, '2017-2018': 1098}

#### Importing Oral Health DataSets Year 2011-2018 

In [10]:
file_paths3 = [
    '/Users/alexandragort/Documents/BA/Data/OralHealth_2011-2012.XPT',
    '/Users/alexandragort/Documents/BA/Data/OralHealth_2013-2014.XPT',
    '/Users/alexandragort/Documents/BA/Data/OralHealth_2015-2016.XPT',
    '/Users/alexandragort/Documents/BA/Data/OralHealth_2017-2018.XPT'
]

oralhealth_data = {}

for path3 in file_paths3:
   
    year_range3 = path3.split('_')[-1].split('.')[0]  # Extracts '2011-2012', '2013-2014', etc.
    
    # Read the dataset
    df = pd.read_sas(path3, format='xport')
    # Defining which columns I want to keep
    columns_to_keep = ['SEQN', 'OHQ030', 'OHQ780A', 'OHQ780B', 'OHQ780C', 'OHQ780D', 'OHQ780E', 'OHQ780F', 'OHQ780G', 'OHQ780H', 'OHQ780I', 'OHQ780J', 'OHQ780K']
    
    df_filtered = df[columns_to_keep]
    
    oralhealth_data[year_range3] = df_filtered

# Example to access data for a specific year range
oralhealth_data['2011-2012'].head()



,SEQN,OHQ030,OHQ780A,OHQ780B,OHQ780C,OHQ780D,OHQ780E,OHQ780F,OHQ780G,OHQ780H,OHQ780I,OHQ780J,OHQ780K
0,62161.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,62162.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,62163.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,62164.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62165.0,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
oralhealth_data['2015-2016'].shape

(9575, 13)

### Creating 4 merged Datasets for all the cycles from 2011-2018:

In [12]:
# Load datasets for each survey cycle for both hic_data and oralhealth_data
hic_data_2011_2012 = hic_data['2011-2012']
hic_data_2013_2014 = hic_data['2013-2014']
hic_data_2015_2016 = hic_data['2015-2016']
hic_data_2017_2018 = hic_data['2017-2018']

oralhealth_data_2011_2012 = oralhealth_data['2011-2012']
oralhealth_data_2013_2014 = oralhealth_data['2013-2014']
oralhealth_data_2015_2016 = oralhealth_data['2015-2016']
oralhealth_data_2017_2018 = oralhealth_data['2017-2018']

def merge_data(df1, df2):
    return pd.merge(df1, df2, on='SEQN', how='outer')

# Merge data for each cycle
merged_2011_2012 = merge_data(hic_data_2011_2012, oralhealth_data_2011_2012)
merged_2013_2014 = merge_data(hic_data_2013_2014, oralhealth_data_2013_2014)
merged_2015_2016 = merge_data(hic_data_2015_2016, oralhealth_data_2015_2016)
merged_2017_2018 = merge_data(hic_data_2017_2018, oralhealth_data_2017_2018)

all_cycles_merged_data = pd.concat([merged_2011_2012, merged_2013_2014, merged_2015_2016, merged_2017_2018])

final_merged_data = merge_data(all_cycles_merged_data, combined_dem_data)

final_merged_data.head()

,SEQN,HIQ011,HIQ031A,HIQ031B,HIQ031C,HIQ031D,HIQ031E,HIQ031F,HIQ031G,HIQ031H,...,SDDSRVYR,RIAGENDR,RIDAGEYR,INDHHIN2,RIDRETH3,DMDCITZN,SDMVPSU,SDMVSTRA,WTINT8YR,WTMEC8YR
0,62161.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,22.0,14.0,3.0,1.0,1.0,91.0,25660.351619,26059.145638
1,62162.0,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,...,7.0,2.0,3.0,4.0,1.0,1.0,3.0,92.0,3864.434224,4029.088502
2,62163.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,14.0,15.0,6.0,1.0,3.0,90.0,1849.421207,1967.371279
3,62164.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,2.0,44.0,8.0,3.0,1.0,1.0,94.0,31837.843325,31991.306551
4,62165.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,2.0,14.0,4.0,4.0,1.0,2.0,90.0,3052.436245,3346.010540


In [13]:
# Renaming the final data set for simplicity
df = final_merged_data

## Data Cleaning

### Handling Missing Values

In [14]:
# Checking for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39156 entries, 0 to 39155
Data columns (total 34 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      39156 non-null  float64
 1   HIQ011    39156 non-null  float64
 2   HIQ031A   17968 non-null  float64
 3   HIQ031B   5385 non-null   float64
 4   HIQ031C   131 non-null    float64
 5   HIQ031D   9488 non-null   float64
 6   HIQ031E   341 non-null    float64
 7   HIQ031F   1028 non-null   float64
 8   HIQ031G   27 non-null     float64
 9   HIQ031H   2403 non-null   float64
 10  HIQ031I   1172 non-null   float64
 11  HIQ031J   1833 non-null   float64
 12  OHQ030    37601 non-null  float64
 13  OHQ780A   4115 non-null   float64
 14  OHQ780B   321 non-null    float64
 15  OHQ780C   1177 non-null   float64
 16  OHQ780D   155 non-null    float64
 17  OHQ780E   164 non-null    float64
 18  OHQ780F   31 non-null     float64
 19  OHQ780G   327 non-null    float64
 20  OHQ780H   279 non-null    fl

In [15]:
df.iloc[:, 20:38].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39156 entries, 0 to 39155
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   OHQ780H   279 non-null    float64
 1   OHQ780I   297 non-null    float64
 2   OHQ780J   240 non-null    float64
 3   OHQ780K   506 non-null    float64
 4   SDDSRVYR  39156 non-null  float64
 5   RIAGENDR  39156 non-null  float64
 6   RIDAGEYR  39156 non-null  float64
 7   INDHHIN2  38106 non-null  float64
 8   RIDRETH3  39156 non-null  float64
 9   DMDCITZN  39142 non-null  float64
 10  SDMVPSU   39156 non-null  float64
 11  SDMVSTRA  39156 non-null  float64
 12  WTINT8YR  39156 non-null  float64
 13  WTMEC8YR  39156 non-null  float64
dtypes: float64(14)
memory usage: 4.2 MB


#### Demography Variables:
Some variabels have either NAN values or 77, 99, 7 or 9 which means they refused to answer or dont know so these will all be categorized as -1 which means basically missing because the values cannot be deleted becuase that would make the weights incorrect.

In [16]:
income_counts = df['INDHHIN2'].value_counts(dropna=False)  
print("Value counts for income:")
print(income_counts)


citizen_counts = df['DMDCITZN'].value_counts(dropna=False)  
print("\nValue counts for citizenship:")
print(citizen_counts)

Value counts for income:
INDHHIN2
15.0    6538
6.0     4278
7.0     3679
14.0    3413
5.0     2823
8.0     2779
4.0     2527
3.0     2304
9.0     2128
10.0    1738
2.0     1504
12.0    1280
1.0     1098
NaN     1050
77.0     894
99.0     592
13.0     531
Name: count, dtype: int64

Value counts for citizenship:
DMDCITZN
1.0    35000
2.0     4065
7.0       59
9.0       18
NaN       14
Name: count, dtype: int64


In [17]:
# Creating a new missing category -1 for citicenship and marital status and income varible:
for column in ['DMDCITZN', 'INDHHIN2']:
    df[column] = df[column].fillna(-1)

df.head()

,SEQN,HIQ011,HIQ031A,HIQ031B,HIQ031C,HIQ031D,HIQ031E,HIQ031F,HIQ031G,HIQ031H,...,SDDSRVYR,RIAGENDR,RIDAGEYR,INDHHIN2,RIDRETH3,DMDCITZN,SDMVPSU,SDMVSTRA,WTINT8YR,WTMEC8YR
0,62161.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,22.0,14.0,3.0,1.0,1.0,91.0,25660.351619,26059.145638
1,62162.0,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,...,7.0,2.0,3.0,4.0,1.0,1.0,3.0,92.0,3864.434224,4029.088502
2,62163.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,14.0,15.0,6.0,1.0,3.0,90.0,1849.421207,1967.371279
3,62164.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,2.0,44.0,8.0,3.0,1.0,1.0,94.0,31837.843325,31991.306551
4,62165.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,2.0,14.0,4.0,4.0,1.0,2.0,90.0,3052.436245,3346.010540


In [18]:
income_counts = df['INDHHIN2'].value_counts(dropna=False)  
print("Value counts for income:")
print(income_counts)

# Get value counts for HIQ031A
citizen_counts = df['DMDCITZN'].value_counts(dropna=False) 
print("\nValue counts for citizenship:")
print(citizen_counts)

Value counts for income:
INDHHIN2
 15.0    6538
 6.0     4278
 7.0     3679
 14.0    3413
 5.0     2823
 8.0     2779
 4.0     2527
 3.0     2304
 9.0     2128
 10.0    1738
 2.0     1504
 12.0    1280
 1.0     1098
-1.0     1050
 77.0     894
 99.0     592
 13.0     531
Name: count, dtype: int64

Value counts for citizenship:
DMDCITZN
 1.0    35000
 2.0     4065
 7.0       59
 9.0       18
-1.0       14
Name: count, dtype: int64


In [19]:
df['INDHHIN2'] = df['INDHHIN2'].replace([77, 99], -1)
df['DMDCITZN'] = df['DMDCITZN'].replace([7, 9], -1)

# Get value counts for INDHHIN2 after replacement
income_counts = df['INDHHIN2'].value_counts(dropna=False)
print("Value counts for income after replacing 77 and 99:")
print(income_counts)

# Get value counts for DMDCITZN after replacement
citizen_counts = df['DMDCITZN'].value_counts(dropna=False)
print("\nValue counts for citizenship after replacing 77 and 99:")
print(citizen_counts)

Value counts for income after replacing 77 and 99:
INDHHIN2
 15.0    6538
 6.0     4278
 7.0     3679
 14.0    3413
 5.0     2823
 8.0     2779
-1.0     2536
 4.0     2527
 3.0     2304
 9.0     2128
 10.0    1738
 2.0     1504
 12.0    1280
 1.0     1098
 13.0     531
Name: count, dtype: int64

Value counts for citizenship after replacing 77 and 99:
DMDCITZN
 1.0    35000
 2.0     4065
-1.0       91
Name: count, dtype: int64


#### Oral Health Variables:

In [20]:
# Check for rows with only nan values of the Oral Health Dummy Variables 
columns_to_check1 = ['OHQ780A', 'OHQ780B', 
                    'OHQ780C', 'OHQ780D', 'OHQ780E', 'OHQ780F', 'OHQ780G', 
                    'OHQ780H', 'OHQ780I', 'OHQ780J', 'OHQ780K']


nan_rows1 = df[columns_to_check1].isna().all(axis=1)

nan_rows_count1 = nan_rows1.sum()

nan_rows_count1

33558

Due to there being so many missing rows this variable (the resaons as to why someone couldn't get dental care) will be dropped. 

In [21]:
df.drop(columns=columns_to_check1, inplace=True)

In [22]:
dental_counts = df['OHQ030'].value_counts(dropna=False)  
print("Value counts for income:")
print(dental_counts)

Value counts for income:
OHQ030
1.0     17255
2.0      5884
3.0      3989
6.0      3708
7.0      2529
4.0      2204
5.0      1938
NaN      1555
99.0       89
77.0        5
Name: count, dtype: int64


In [23]:
for column in ['OHQ030']:
    df[column] = df[column].fillna(-1)

df.head()

,SEQN,HIQ011,HIQ031A,HIQ031B,HIQ031C,HIQ031D,HIQ031E,HIQ031F,HIQ031G,HIQ031H,...,SDDSRVYR,RIAGENDR,RIDAGEYR,INDHHIN2,RIDRETH3,DMDCITZN,SDMVPSU,SDMVSTRA,WTINT8YR,WTMEC8YR
0,62161.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,22.0,14.0,3.0,1.0,1.0,91.0,25660.351619,26059.145638
1,62162.0,1.0,NaN,NaN,NaN,17.0,NaN,NaN,NaN,NaN,...,7.0,2.0,3.0,4.0,1.0,1.0,3.0,92.0,3864.434224,4029.088502
2,62163.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,1.0,14.0,15.0,6.0,1.0,3.0,90.0,1849.421207,1967.371279
3,62164.0,1.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,2.0,44.0,8.0,3.0,1.0,1.0,94.0,31837.843325,31991.306551
4,62165.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,2.0,14.0,4.0,4.0,1.0,2.0,90.0,3052.436245,3346.010540


In [24]:
df['OHQ030'] = df['OHQ030'].replace([77, 99], -1)


# Get value counts for INDHHIN2 after replacement
dental_counts = df['OHQ030'].value_counts(dropna=False)
print("Value counts for income after replacing 77 and 99:")
print(dental_counts)

Value counts for income after replacing 77 and 99:
OHQ030
 1.0    17255
 2.0     5884
 3.0     3989
 6.0     3708
 7.0     2529
 4.0     2204
 5.0     1938
-1.0     1649
Name: count, dtype: int64


In [25]:
df.shape
# we still kept the original shape (for the rows which is important because of the weights :) )

(39156, 23)

#### Health Insurance Coverage Variables:

In [26]:
# Check for rows with only nan values in the Health Insurance Coverage Dummy Variables that also actually have insurance
columns_to_check2 = ['HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E', 
                    'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'HIQ031J']

nan_rows_with_insurance = df[df[columns_to_check2].isna().all(axis=1) & (df['HIQ011'] == 1.0)]


nan_rows_with_insurance.shape[0]

20

In [27]:
nan_rows_with_insurance = df[df[columns_to_check2].isna().all(axis=1) & (df['HIQ011'] == 1.0)]

# if someone actually is said to have insurance but then its not specified as which i also classify them as missing
df.loc[df[columns_to_check2].isna().all(axis=1) & (df['HIQ011'] == 1.0), 'HIQ011'] = -1


In [28]:
columns_to_check2 = ['HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E', 
                     'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'HIQ031J']

# Loop through each column and print the value counts
for column in columns_to_check2:
    print(f"Value counts for {column}:")
    print(df[column].value_counts(dropna=False))  # Include NaN in the counts
    print("\n")  # Add a newline for better readability


Value counts for HIQ031A:
HIQ031A
NaN     21188
14.0    17766
99.0      193
77.0        9
Name: count, dtype: int64


Value counts for HIQ031B:
HIQ031B
NaN     33771
15.0     5385
Name: count, dtype: int64


Value counts for HIQ031C:
HIQ031C
NaN     39025
16.0      131
Name: count, dtype: int64


Value counts for HIQ031D:
HIQ031D
NaN     29668
17.0     9488
Name: count, dtype: int64


Value counts for HIQ031E:
HIQ031E
NaN     38815
18.0      341
Name: count, dtype: int64


Value counts for HIQ031F:
HIQ031F
NaN     38128
19.0     1028
Name: count, dtype: int64


Value counts for HIQ031G:
HIQ031G
NaN     39129
20.0       27
Name: count, dtype: int64


Value counts for HIQ031H:
HIQ031H
NaN     36753
21.0     2403
Name: count, dtype: int64


Value counts for HIQ031I:
HIQ031I
NaN     37984
22.0     1172
Name: count, dtype: int64


Value counts for HIQ031J:
HIQ031J
NaN     37323
23.0     1833
Name: count, dtype: int64




In [29]:
columns_to_check2 = ['HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E', 
                     'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'HIQ031J']

# Replace NaN values with 0, 77 and 99 with -1
df[columns_to_check2] = df[columns_to_check2].applymap(
    lambda x: -1 if x in [77, 99] else (0 if pd.isna(x) else x)
)


/var/folders/rn/hshtt4vd17q2vswjbp95szmr0000gn/T/ipykernel_97883/241112286.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[columns_to_check2] = df[columns_to_check2].applymap(


In [30]:
columns_to_check2 = ['HIQ031A', 'HIQ031B', 'HIQ031C', 'HIQ031D', 'HIQ031E', 
                     'HIQ031F', 'HIQ031G', 'HIQ031H', 'HIQ031I', 'HIQ031J']

# Loop through each column and print the value counts
for column in columns_to_check2:
    print(f"Value counts for {column}:")
    print(df[column].value_counts(dropna=False))  # Include NaN in the counts
    print("\n")  # Add a newline for better readability


Value counts for HIQ031A:
HIQ031A
 0.0     21188
 14.0    17766
-1.0       202
Name: count, dtype: int64


Value counts for HIQ031B:
HIQ031B
0.0     33771
15.0     5385
Name: count, dtype: int64


Value counts for HIQ031C:
HIQ031C
0.0     39025
16.0      131
Name: count, dtype: int64


Value counts for HIQ031D:
HIQ031D
0.0     29668
17.0     9488
Name: count, dtype: int64


Value counts for HIQ031E:
HIQ031E
0.0     38815
18.0      341
Name: count, dtype: int64


Value counts for HIQ031F:
HIQ031F
0.0     38128
19.0     1028
Name: count, dtype: int64


Value counts for HIQ031G:
HIQ031G
0.0     39129
20.0       27
Name: count, dtype: int64


Value counts for HIQ031H:
HIQ031H
0.0     36753
21.0     2403
Name: count, dtype: int64


Value counts for HIQ031I:
HIQ031I
0.0     37984
22.0     1172
Name: count, dtype: int64


Value counts for HIQ031J:
HIQ031J
0.0     37323
23.0     1833
Name: count, dtype: int64




In [31]:
df['HIQ011'] = df['HIQ011'].replace([7, 9], -1)


# Get value counts for HIQ011 after replacement
HI_counts = df['HIQ011'].value_counts(dropna=False)
print("Value counts for income after replacing 77 and 99:")
print(HI_counts)

Value counts for income after replacing 77 and 99:
HIQ011
 1.0    33404
 2.0     5657
-1.0       95
Name: count, dtype: int64


#### Final Touches

In [32]:
# Replace NaN values with 0 
df = df.fillna(0.0)


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39156 entries, 0 to 39155
Data columns (total 23 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   SEQN      39156 non-null  float64
 1   HIQ011    39156 non-null  float64
 2   HIQ031A   39156 non-null  float64
 3   HIQ031B   39156 non-null  float64
 4   HIQ031C   39156 non-null  float64
 5   HIQ031D   39156 non-null  float64
 6   HIQ031E   39156 non-null  float64
 7   HIQ031F   39156 non-null  float64
 8   HIQ031G   39156 non-null  float64
 9   HIQ031H   39156 non-null  float64
 10  HIQ031I   39156 non-null  float64
 11  HIQ031J   39156 non-null  float64
 12  OHQ030    39156 non-null  float64
 13  SDDSRVYR  39156 non-null  float64
 14  RIAGENDR  39156 non-null  float64
 15  RIDAGEYR  39156 non-null  float64
 16  INDHHIN2  39156 non-null  float64
 17  RIDRETH3  39156 non-null  float64
 18  DMDCITZN  39156 non-null  float64
 19  SDMVPSU   39156 non-null  float64
 20  SDMVSTRA  39156 non-null  fl

Great looks like we've filled all the NAN-values and the data types are all the same, and no rows were dropped.

### Creating new Variable Names

In [34]:
# Creating new variable names to make it more easily readable
rename_columns = {
    'SEQN': 'seqn',
    'SDDSRVYR': 'releaseyear',
    'RIDAGEYR': 'age',
    'RIAGENDR': 'gender',
    'RIDRETH3': 'race',
    'DMDCITZN': 'citizenship',
    'DMDMARTL': 'maritalstatus',
    'INDHHIN2': 'income',
    'HIQ011': 'HI_or_Not',  
    'HIQ031A': 'Private',  # PRIVATE HEALTH INSURANCE
    'HIQ031B': 'Medicare',  # MEDICAR
    'HIQ031C': 'Medi-Gap',  # MEDI-GAP
    'HIQ031D': 'Medicaid',  # MEDICAID
    'HIQ031E': 'CHIP',  # SCHIP (CHIP/CHILDREN’S HEALTH INSURANCE PROGRAM)
    'HIQ031F': 'Military',  # MILITARY HEALTHCARE (TRICARE/VA/CHAMP-VA)
    'HIQ031G': 'Indian',  # INDIAN HEALTH SERVICE
    'HIQ031H': 'State',  # STATE-SPONSORED HEALTH PLAN
    'HIQ031I': 'Other',  # OTHER GOVERNMENT PROGRAM
    'HIQ031J': 'Single_Service',  # SINGLE SERVICE PLAN (E.G., DENTAL, VISION, PRESCRIPTIONS)
    'OHQ030': 'Dental_Visits',
}
df.rename(columns=rename_columns, inplace=True)


df


,seqn,HI_or_Not,Private,Medicare,Medi-Gap,Medicaid,CHIP,Military,Indian,State,...,releaseyear,gender,age,income,race,citizenship,SDMVPSU,SDMVSTRA,WTINT8YR,WTMEC8YR
0,62161.0,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,1.0,22.0,14.0,3.0,1.0,1.0,91.0,25660.351619,26059.145638
1,62162.0,1.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,...,7.0,2.0,3.0,4.0,1.0,1.0,3.0,92.0,3864.434224,4029.088502
2,62163.0,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,1.0,14.0,15.0,6.0,1.0,3.0,90.0,1849.421207,1967.371279
3,62164.0,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,2.0,44.0,8.0,3.0,1.0,1.0,94.0,31837.843325,31991.306551
4,62165.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,2.0,14.0,4.0,4.0,1.0,2.0,90.0,3052.436245,3346.010540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39151,102952.0,1.0,14.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,2.0,70.0,4.0,6.0,1.0,2.0,138.0,4224.069051,4584.677776
39152,102953.0,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,1.0,42.0,12.0,1.0,2.0,2.0,137.0,15407.595003,15915.487893
39153,102954.0,1.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,...,10.0,2.0,41.0,10.0,4.0,1.0,1.0,144.0,4290.223817,4423.695836
39154,102955.0,1.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,2.0,14.0,9.0,4.0,1.0,1.0,136.0,3559.611481,3717.959909


Now I will recode all the dummy variables so that they are easier to work with:

In [35]:
value_counts_hi_or_not = df['HI_or_Not'].value_counts()
print(value_counts_hi_or_not)

HI_or_Not
 1.0    33404
 2.0     5657
-1.0       95
Name: count, dtype: int64


In [36]:
df['HI_or_Not'] = df['HI_or_Not'].replace(2.0, 0)
value_counts_hi_or_not = df['HI_or_Not'].value_counts()
print(value_counts_hi_or_not)

HI_or_Not
 1.0    33404
 0.0     5657
-1.0       95
Name: count, dtype: int64


In [37]:
df['Private'] = df['Private'].replace(14.0, 1.0)
value_counts_private = df['Private'].value_counts()
print(value_counts_private)

Private
 0.0    21188
 1.0    17766
-1.0      202
Name: count, dtype: int64


In [38]:
df['Medicare'] = df['Medicare'].replace(15.0, 1.0)
value_counts_medicare = df['Medicare'].value_counts()
print(value_counts_medicare)

Medicare
0.0    33771
1.0     5385
Name: count, dtype: int64


In [39]:
df['Medi-Gap'] = df['Medi-Gap'].replace(16.0, 1.0)
value_counts_medigap = df['Medi-Gap'].value_counts()
print(value_counts_medigap)

Medi-Gap
0.0    39025
1.0      131
Name: count, dtype: int64


In [40]:
df['Medicaid'] = df['Medicaid'].replace(17.0, 1.0)
value_counts_medicaid = df['Medicaid'].value_counts()
print(value_counts_medicaid)

Medicaid
0.0    29668
1.0     9488
Name: count, dtype: int64


In [41]:
df['CHIP'] = df['CHIP'].replace(18.0, 1.0)
value_counts_chip = df['CHIP'].value_counts()
print(value_counts_chip)

CHIP
0.0    38815
1.0      341
Name: count, dtype: int64


In [42]:
df['Military'] = df['Military'].replace(19.0, 1.0)
value_counts_military = df['Military'].value_counts()
print(value_counts_military)

Military
0.0    38128
1.0     1028
Name: count, dtype: int64


In [43]:
df['Indian'] = df['Indian'].replace(20.0, 1.0)
value_counts_indian = df['Indian'].value_counts()
print(value_counts_indian)

Indian
0.0    39129
1.0       27
Name: count, dtype: int64


In [44]:
df['State'] = df['State'].replace(21.0, 1.0)
value_counts_state = df['State'].value_counts()
print(value_counts_state)

State
0.0    36753
1.0     2403
Name: count, dtype: int64


In [45]:
df['Other'] = df['Other'].replace(22.0, 1.0)
value_counts_other = df['Other'].value_counts()
print(value_counts_other)

Other
0.0    37984
1.0     1172
Name: count, dtype: int64


In [46]:
df['Single_Service'] = df['Single_Service'].replace(23.0, 1.0)
value_counts_single = df['Single_Service'].value_counts()
print(value_counts_single)

Single_Service
0.0    37323
1.0     1833
Name: count, dtype: int64


Now I also wanna create an additional variable which will be a dummy variable for the dental_visits variable where I have one category with frequent visits == 1 (visited some dental service within the last year) and a non frequent category == 0 (visited more than a year ago) that I can also use fore the analysis. I will call it dental_visits_dummy:

In [47]:
df['Dental_Dummy'] = df['Dental_Visits'].apply(lambda x: 1 if x in [1, 2] else 0)
value_counts_dental_visits_dummy = df['Dental_Dummy'].value_counts()
print(value_counts_dental_visits_dummy)


Dental_Dummy
1    23139
0    16017
Name: count, dtype: int64


I also wanna create a varibale called low_income where people with inocme below

In [48]:
df['income'].value_counts()

income
 15.0    6538
 6.0     4278
 7.0     3679
 14.0    3413
 5.0     2823
 8.0     2779
-1.0     2536
 4.0     2527
 3.0     2304
 9.0     2128
 10.0    1738
 2.0     1504
 12.0    1280
 1.0     1098
 13.0     531
Name: count, dtype: int64

In [49]:
low_income_categories = [1, 2, 3, 4, 5, 6, 13]

# missing values should stay -1 though
df['low_income'] = df['income'].apply(
    lambda x: -1 if x == -1 else (1 if x in low_income_categories else 0)
)

value_counts_low_income = df['low_income'].value_counts(dropna=False)
print(value_counts_low_income)

low_income
 0    21555
 1    15065
-1     2536
Name: count, dtype: int64


In [50]:
df

,seqn,HI_or_Not,Private,Medicare,Medi-Gap,Medicaid,CHIP,Military,Indian,State,...,age,income,race,citizenship,SDMVPSU,SDMVSTRA,WTINT8YR,WTMEC8YR,Dental_Dummy,low_income
0,62161.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,22.0,14.0,3.0,1.0,1.0,91.0,25660.351619,26059.145638,1,0
1,62162.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.0,4.0,1.0,1.0,3.0,92.0,3864.434224,4029.088502,0,1
2,62163.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,15.0,6.0,1.0,3.0,90.0,1849.421207,1967.371279,1,0
3,62164.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44.0,8.0,3.0,1.0,1.0,94.0,31837.843325,31991.306551,1,0
4,62165.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,4.0,4.0,1.0,2.0,90.0,3052.436245,3346.010540,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39151,102952.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,70.0,4.0,6.0,1.0,2.0,138.0,4224.069051,4584.677776,1,1
39152,102953.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,42.0,12.0,1.0,2.0,2.0,137.0,15407.595003,15915.487893,0,0
39153,102954.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,41.0,10.0,4.0,1.0,1.0,144.0,4290.223817,4423.695836,0,0
39154,102955.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,9.0,4.0,1.0,1.0,136.0,3559.611481,3717.959909,1,0


In [51]:
count_minus_one_rows = (df == -1).any(axis=1).sum()

print(f"Number of rows containing -1: {count_minus_one_rows}")

Number of rows containing -1: 4397


out of all the 39156 rows we have a total of 4397 with missing values, which will be subsetted and omitted later before doing the analysis.

Now I will save my df as a csv to import it into R to employ the survey package!

In [52]:
df.to_csv('/Users/alexandragort/Documents/BA/BA_df.csv', index=False)


Now we finally have everything clean and no more missing values! :-)
Ready to perform the analysis!